# Simplify symbolic Racah algebra expressions 

In [1]:
using JAC, SymEngine

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11208817502857679691\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12720506943503241037\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11208817502857679691\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

**Note**: The Julia package `SymEngine` is needed to perform symbolic simplifications of Racah algebra expressions in JAC but, by default, is not automatically loaded.


In atomic and quantum many-particle physics, (the coupling of) angular momenta and spherical tensors play a crucial role for the the evaluation and spin-angular integration of many-particle matrix elements. This coupling often leads to algebraic expressions which are usually written in terms of generalized Clebsch-Gordan coefficients and/or Wigner 3n–j symbols as well as the Wigner rotation matrices and spherical harmonics. Although the evaluation and simplification of such expressions is, at least *in principle*, a rather straigthforward task, it may become (very) cumbersome if complex systems or physical scenarios are considered.

After the pioneering work by Wigner in the late 1930s, in particular Guilio Racah developed a powerful machinery, known as **Racah algebra (techniques)** today, in order to deal with the angular momentum of (rotational invariant) quantum many-particle systems. Briefly speaking, this machinery includes a number of strategies to simplify (so-called) **Racah expressions**. In JAC, these expressions are defined by a summation over Wigner n-j symbols of different kind(s) as well as (various integrals over) the spherical harmonics and Kronecker and triangular deltas, cf. User Guide, Section 15.1. Here, the summation may formally run over any number of angular momenta. Since, in the theory of angular momentum, most symbols obey a rather high symmetry, the complexity of such Racah expressions increases very rapidly as more Wigner symbols, rotation matrices and or spherical harmonics appear in the (product) terms over which one needs to sum.

There are different strategies, that can be (successively) applied to simplify such Racah expressions algebracially, i.e. if the angular momenta are not specified numerically. These strategies include:

+ Use of known special values. This replaces a Wigner 3n-jsymbol or spherical harmomic by a (much) simpler expression that, in particular, does not contain implicit summation. In practice, each Wigner 3n-j symbol can be analysed and perhaps replaced independently by some special-value rule.
+ Use of orthogonality relations for the Wigner symbols or spherical harmonics.
+ Use of known algebraic sum rules for the Wigner symbols

To make use of these strategies, we need a suitable representation of the various symbols from the theory of angular momenta as well as for Racah expressions as a whole. Therefore, let us first have a look for the (internal) representation of a Wigner 3-j symbol `W3j(ja,jb,jc,ma,mb,mc)`

In [2]:
?W3j

search: W3j



`struct  RacahAlgebra.W3j`  ... defines a type for a Wigner 3-j symbol with symbolic arguments.

```
+ ja, jb, jc     ::Basic   ... angular momenta
+ ma, mb, mc     ::Basic   ... projections of the angular momenta above
```

---

`RacahAlgebra.W3j(ja::AngMomentum, jb::AngMomentum, jc::AngMomentum, ma::AngMomentum, mb::AngMomentum, mc::AngMomentum)`       ... constructor for defining the Wigner 3-j symbol either by Julia Symbol's or SymEngine Basic variables.


which takes six arguments of the (internal) type `AngMomentum`

In [3]:
?RacahAlgebra.AngMomentum

`struct RacahAlgebra.AngMomentum`      ... defines an (abstract) data types for symbolic angular momenta which accept the types          Basic, Symbol, Int64 and Rational{Int64} and check for being consistent with angular momenta.


A slightly more complex struct, we have for a `RacahExpression` as a whole:

In [4]:
?RacahExpression

search: RacahExpression



`struct  RacahAlgebra.RacahExpression`  ... defines a type for a RacahExpression with symbolic arguments.

```
+ summations    ::Array{Basic,1}      ... Summation indices.
+ phase         ::Basic               ... Phase of the Racah expression.
+ weight        ::Basic               ... Weight of the Racah expression.
+ deltas        ::Array{Kronecker,1}  ... List of Kronecker deltas.
+ triangles     ::Array{Triangle,1}   ... List of Triangle deltas.
+ w3js          ::Array{W3j,1}        ... List of Wigner 3-j symbols
+ w6js          ::Array{W6j,1}        ... List of Wigner 6-j symbols
+ w9js          ::Array{W9j,1}        ... List of Wigner 9-j symbols
```

---

`RacahAlgebra.RacahExpression()`       ... constructor for defining an empty RacahExpression.


In order to make use of JAC's extension for simplifying expressions from Racah' algebra, we shall need also a number of symbolic variables, whose type `Basic` is derived from `SymEngine` (although not much in-line documentation is provided by this package):

In [5]:
?Basic

search: Basic Basics Basis UseBabushkin AbstractString



No documentation found.

# Summary

```
mutable struct Basic <: Number
```

# Fields

```
ptr :: Ptr{Nothing}
```

# Supertype Hierarchy

```
Basic <: Number <: Any
```


We define a number of such (symbolic) variables to facilitate our later discussion:

In [6]:
a  = Basic(:a);     b  = Basic(:b);    c  = Basic(:c);   d  = Basic(:d);     ee  = Basic(:ee);    f  = Basic(:f)
X  = Basic(:X);     Y  = Basic(:Y);    Z  = Basic(:Z)  
j  = Basic(:j);     m = Basic(:m);   typeof(m)

Basic

Perhaps the simple way in looking for simplifications of either general Racah expressions or individual Wigner symbols is to (try to) detect special values of the Wigner n-j symbols. For instance, we may consider:

In [7]:
w3j = W3j(j+3//2, j, 3//2, m, -m-3//2, 3//2)

W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2)

We can try to simplify these Wigner symbol by:

In [8]:
wa = RacahAlgebra.evaluate(w3j)

** Special value found for  W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2) = (-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))   


(true, (-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  )

The result of such an evaluation is a `Tuple{Bool,RacahExpression}`, and where the first entry shows of whether a simplification was found (true), while the second entry contains the corresponding Racah expression. An (empty) Racah expression is returned if no simplification is found. We can display the result by:

In [9]:
wa[2]

(-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  

Alternatively, we may call this evaluation as part of a `RacahExpression` as in the following example:

In [10]:
w6j = W6j( a, b, c, 2, c-2, b-2)
rex = RacahAlgebra.equivalentForm(w6j)

** Select 16th equivalent form for W6j{a, b, c; 2, -2 + c, -2 + b}    ==>   W6j{-2 + c, c, 2; b, -2 + b, a}  


W6j{-2 + c, c, 2; b, -2 + b, a}  

After the definition of the Wigner 6-j symbol, we here invoke randomly one of its 24 **symmetric forms** of the 6-j symbols. The type of rex is no longer `W6j` but `RacahExpression` since, most generally, different symmetric forms of the Wigner n-j symbols will lead to additional phase factors.

In [11]:
typeof(rex)

RacahExpression

We can simplify this Racah expression by:

In [12]:
wb  = RacahAlgebra.evaluate(rex, special=true)

** Special value found for  W6j{-2 + c, c, 2; b, -2 + b, a} = (-1)^(a + b + c)  ((1/2)*sqrt((-1 + a + b + c)*(a + b + c)*(-2 + a + b + c)*(-3 - a + b + c)*(-2 - a + b + c)*(-1 - a + b + c)*(-a + b + c)/(b*c*(-3 + 2*b)*(-2 + 2*b)*(1 + 2*b)*(-1 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))   


(-1)^(a + b + c)  ((1/2)*sqrt((-1 + a + b + c)*(a + b + c)*(-2 + a + b + c)*(-3 - a + b + c)*(-2 - a + b + c)*(-1 - a + b + c)*(-a + b + c)/(b*c*(-3 + 2*b)*(-2 + 2*b)*(1 + 2*b)*(-1 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))  

In contrast to the literature, where the special values are typically shown for one *standard form* of the Wigner symbols (as, for instance, in the initial definition above), JAC finds the special value in all cases as it internally *cycles* through all symmetries, keeping the corresponding phase and weight factors (cf. later) into account. The type of `wb` is still a `RacahExpression`, although it is printed here in a -- more or less -- neat format.

In [13]:
typeof(wb)

RacahExpression

Apart from the classical symmetries, the (so-called) *Regge symmetries* are known for the Wigner 3-j and 6-j symbols are known and could be readily implemented, if necessary. In practice, various **special values of the Wigner n-j** are recognized by Mathematica and, perhaps, other computer-algebra systems. We include them here for completeness, while the **main emphasis** is placed upon the **sum rule evaluation**, a very special and convenient feature of JAC.

Typically, only some **standard form of each sum rule** is shown in the literature, and many of these sum rules are displayed only in some specialized literature about angular momenta. Likely, the most comprehensive compilation of these (and many other) rules can be found in the monograph by Varshalovich et al. (1988). --- In general, however, one needs to recognize all the symmetries of a Racah expressions, implying all the phases and possible (weight) factors that arise from these symmetries. In JAC, this is realized by cycling automatically through all symmetric forms of the Wigner n-j (n = 3,6,9) symbols. In a later step, we also plan to take the spherical harmonics and the Wigner rotation matrices into account as well.

**Until the present, however, only a rather restricted set of sum rules have been implemented and tested in detail.** 

Let us demonstrate the basic features of the JAC tools by first looking at a sum rule for the Wigner 9-j symbols:

In [14]:
w9j = W9j(a, b, ee, c, d, f, ee, f, X)
rex = RacahExpression( [X], Basic(0), Basic(2*X+1), Kronecker[], Triangle[], W3j[], W6j[], [w9j] )

Sum_[Basic[X]]  (1 + 2*X)  W9j{a, b, ee; c, d, f; ee, f, X}  

In [15]:
rex = RacahAlgebra.equivalentForm(rex)

** Select 71th equivalent form for W9j{a, b, ee; c, d, f; ee, f, X}    ==>   W9j{X, f, ee; f, d, c; ee, b, a}  


Sum_[Basic[X]]  (1 + 2*X)  W9j{X, f, ee; f, d, c; ee, b, a}  

As before, we can try to simplify this expression by `RacahAlgebra.evaluate(rex)`; alternatively, we could skip the last step and evalute the `rex` directly:

In [16]:
RacahAlgebra.evaluate(rex)

+++ rex.phase = 0   xaRex.phase = 0
** Apply sum rule for one W9j -- Sum(X) ....


((1 + 2*c)^(-1))  delta(c, b)  delta (d, c, f)  delta (c, a, ee)  

Apparently, the summation over the Wigner 9-j symbols, and including the formal angular momentum X, can be replaced by just one Kronecker delta [delta(a,b)] as well as two (so-called) *triangular deltas* [delta(a,b,c)], which are 1 if tje lengths a,b,c may form a triangle and zero otherwise. These *triangular deltas* are quite well-known in the theory of angular momentum to indicate a valid coupling of two angular momenta [ja + jb] --> |ja-jb|, |ja-jb|+1, ..., ja+jb.

In another example, we consider a sum rule for two Wigner 6-j symbols:

In [17]:
aw6j = W6j(X, Y, Z, a, b ,c);    bw6j = W6j(X, Y, Z, a, b ,c)
rex = RacahExpression( [X, Y, Z], Basic(0), Basic((2*X+1)*(2*Y+1)*(2*Z+1)), Kronecker[], Triangle[], 
                       W3j[], W6j[aw6j, bw6j], W9j[] )

Sum_[Basic[X, Y, Z]]  ((1 + 2*Y)*(1 + 2*Z)*(1 + 2*X))  W6j{X, Y, Z; a, b, c}  W6j{X, Y, Z; a, b, c}  

and try to simplify it as usual:

In [18]:
RacahAlgebra.evaluate(rex)

** Apply sum rule for two W6j -- Sum(X,Y,Z) [X,Y,Z] ...


delta (X, Y, Z)  

Of course, such simplifications does work only for very selective expressions; if we just *replace* one of the summation variables or the *phase/factor* of this expression, no further simplification is found (and likely possible in generally):

In [19]:
aw6j = W6j(X, Y, d, a, b ,c);    bw6j = W6j(X, Y, Z, a, b ,c)
rex = RacahExpression( [X, Y, f], Basic(0), Basic((2*X+1)*(2*Y+1)*(2*Z+1)), Kronecker[], Triangle[], 
                       W3j[], W6j[aw6j, bw6j], W9j[] )
RacahAlgebra.evaluate(rex)


No simplification found for:  Sum_[Basic[X, Y, f]]  ((1 + 2*Y)*(1 + 2*Z)*(1 + 2*X))  W6j{X, Y, d; a, b, c}  W6j{X, Y, Z; a, b, c}   


In the future, we shall implement further rules though this might also depend on the interest from the user side.

# **The following part of this nootebook is still under construction.**